## Big data
Hipótese a ser provada: Cidades que possuem um PIB per capita maior possuem maiores emissões de GEE.
- análise do aumento das emissões conforme variação do PIB per capita
- análise das cidades com maior crescimento nas emissões de GEE
- análise dos setores com maior crescimento nas emissões de GEE
- análise dos setores com maior redução nas emissões de GEE --> relação com crescimmento das cidades

In [ ]:
#%pip install duckdb

In [ ]:
# Célula 1
import duckdb
import pandas as pd
from pathlib import Path

PARQUET_PATH = Path("/Users/pedro/Desktop/Big Data/emissoes_gee/todas_emissoes.parquet")
df = pd.read_parquet(PARQUET_PATH)
conn = duckdb.connect(database=':memory:')
conn.register('emissoes', df)

In [ ]:
# ve quais colunas eu tenho
conn.execute("PRAGMA table_info('emissoes');").df()

In [ ]:
# ve quanto de emissão total por década
conn.execute("""
SELECT 
    decada,
    SUM(emissao_t) as emissao_total
FROM emissoes
GROUP BY decada
ORDER BY decada
""").df()

In [ ]:
# quanto cada setor emitiu
conn.execute("""
SELECT 
    Setor_Agrupado,
    SUM(emissao_t) as emissao_total,
    ROUND(SUM(emissao_t) * 100.0 / (SELECT SUM(emissao_t) FROM emissoes), 2) as percentual
FROM emissoes
GROUP BY Setor_Agrupado
ORDER BY emissao_total DESC
LIMIT 10
""").df()

In [ ]:
# ver o quanto cada setor emitiu em cada decada
conn.execute("""
SELECT 
    decada,
    Setor_Agrupado,
    SUM(emissao_t) as emissao_total,
    ROUND(SUM(emissao_t) * 100.0 / SUM(SUM(emissao_t)) OVER (PARTITION BY decada), 2) as percentual_na_decada
FROM emissoes
GROUP BY Setor_Agrupado, decada
ORDER BY Setor_Agrupado, decada, emissao_total DESC
""").df()

In [ ]:
# ver o quanto cada setor emitiu em cada decada, ordenado pela decada e emissao total
conn.execute("""
SELECT 
    decada,
    Setor_Agrupado,
    SUM(emissao_t) as emissao_total
FROM emissoes
GROUP BY decada, Setor_Agrupado
ORDER BY decada, emissao_total DESC
""").df()

In [ ]:
# variação por setor
conn.execute("""
WITH primeira AS (
    SELECT MIN(decada) as dec FROM emissoes
),
ultima AS (
    SELECT MAX(decada) as dec FROM emissoes
)
SELECT 
    Setor_Agrupado,
    ROUND(
        (SUM(CASE WHEN e.decada = u.dec THEN emissao_t ELSE 0 END) - 
         SUM(CASE WHEN e.decada = p.dec THEN emissao_t ELSE 0 END)) * 100.0 /
        NULLIF(SUM(CASE WHEN e.decada = p.dec THEN emissao_t ELSE 0 END), 0),
        2
    ) as variacao_percentual
FROM emissoes e
CROSS JOIN primeira p
CROSS JOIN ultima u
GROUP BY Setor_Agrupado
ORDER BY variacao_percentual DESC
""").df()

In [ ]:
# evolução das emissões totais por década
conn.execute("""
SELECT 
    decada,
    SUM(emissao_t) as emissao_total,
FROM emissoes
GROUP BY decada
ORDER BY decada
""").df()

In [ ]:
# variação por década
conn.execute("""
WITH primeira AS ( 
    SELECT MIN(decada) as dec FROM emissoes
),
ultima AS (
    SELECT MAX(decada) as dec FROM emissoes
)
SELECT 
    decada,
    ROUND(
        (SUM(CASE WHEN e.decada = u.dec THEN emissao_t ELSE 0 END) - 
         SUM(CASE WHEN e.decada = p.dec THEN emissao_t ELSE 0 END)) * 100.0 /
        NULLIF(SUM(CASE WHEN e.decada = p.dec THEN emissao_t ELSE 0 END), 0),
        2
    ) as variacao_percentual
FROM emissoes e
CROSS JOIN primeira p
CROSS JOIN ultima u
GROUP BY decada
ORDER BY variacao_percentual DESC
""").df()

In [ ]:
# ve quais as 20 cidades que mais emitiram no total
conn.execute("""
SELECT 
    Cidade,
    SUM(emissao_t) as emissao_total,
FROM emissoes
GROUP BY Cidade
ORDER BY emissao_total DESC
LIMIT 20
""").df()

In [ ]:
# municípios com maior crescimento
conn.execute("""
WITH primeira AS (SELECT MIN(decada) as dec FROM emissoes),
     ultima AS (SELECT MAX(decada) as dec FROM emissoes)
SELECT 
    Cidade,
    SUM(CASE WHEN e.decada = u.dec THEN emissao_t ELSE 0 END) - 
    SUM(CASE WHEN e.decada = p.dec THEN emissao_t ELSE 0 END) as variacao
FROM emissoes e
CROSS JOIN primeira p
CROSS JOIN ultima u
WHERE Cidade IS NOT NULL
GROUP BY Cidade
ORDER BY variacao DESC
LIMIT 10
""").df()

In [ ]:
# emissões por bioma
conn.execute("""
SELECT 
    Bioma,
    SUM(emissao_t) as emissao_total,
    ROUND(SUM(emissao_t) * 100.0 / (SELECT SUM(emissao_t) FROM emissoes WHERE Bioma IS NOT NULL), 2) as percentual
FROM emissoes
WHERE Bioma IS NOT NULL
GROUP BY Bioma
ORDER BY emissao_total DESC
""").df()

In [ ]:
# tipo de gás
conn.execute("""
SELECT 
    "Gás",
    SUM(emissao_t) as emissao_total,
    ROUND(AVG(emissao_t), 2) as emissao_media,
FROM emissoes
WHERE "Gás" IS NOT NULL
GROUP BY "Gás"
ORDER BY emissao_total DESC
""").df()